here original data is used to truncate the lists of time series columns and unlike previous attempts, failure columns are kept to be used as categorical columns.

In [1]:
import pandas as pd
import numpy as np

In [2]:
original_data = pd.read_parquet("../../data/test_107_reduced.parquet")
original_data

,doorcyclecounter,doorforce,doorspeed,doorposition,cd_counter_rollers_installation,ld_counter_rollers_installation,closing_device_failure,misalignment_cd_vs_ld,misalignment_cd_vs_ld_sill_gap,belt_tension,pulley_is_touching_belt,zero_position,cmcouplerfriction,cmdoorfriction,cmelectronicage,cmvibration
0,1166,"[-75.1, -75.1, -92.8, -73.5, -51.9, -46.3, -46...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0531, 0.078...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0451, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,154.899994,121.300003,0.86,0.9
1,1167,"[-75.1, -75.1, -85.0, -92.1, -52.8, -51.6, -51...","[0.0016, 0.0016, 0.0016, 0.0016, 0.033, 0.0822...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.048, -...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,134.399994,120.699997,0.85,0.7
2,1168,"[-75.1, -75.1, -87.9, -88.3, -61.2, -51.1, -51...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0612, 0.086...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0468, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.600006,119.800003,0.85,0.9
3,1169,"[-75.1, -75.1, -85.6, -83.4, -57.2, -51.0, -51...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0505, 0.087...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0474, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,145.800003,122.000000,0.86,0.8
4,1170,"[-75.1, -75.1, -78.5, -91.4, -59.5, -49.7, -49...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0297, 0.073...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0483, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.100006,120.400002,0.85,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4875,6657,"[-75.1, -75.1, -93.3, -59.5, -38.7, -29.9, -29...","[0.0016, 0.0016, 0.0016, 0.0016, 0.054, 0.0706...","[-0.0424, -0.0424, -0.0424, -0.0424, -0.0385, ...",0.0,0.0,False,0.0,0.0,0.0,False,0.0,127.199997,120.000000,0.87,0.9
4876,6658,"[-75.1, -75.1, -90.3, -87.9, -55.8, -47.9, -47...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0462, 0.078...","[-0.0424, -0.0424, -0.0424, -0.0424, -0.0401, ...",0.0,0.0,False,0.0,0.0,0.0,False,0.0,130.899994,111.699997,0.86,1.0
4877,6659,"[-75.1, -75.1, -90.5, -78.1, -52.6, -49.0, -49...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0475, 0.081...","[-0.0424, -0.0424, -0.0424, -0.0424, -0.0397, ...",0.0,0.0,False,0.0,0.0,0.0,False,0.0,129.500000,120.900002,0.89,1.0
4878,6660,"[-75.1, -75.1, -89.8, -82.6, -53.0, -49.3, -49...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0543, 0.082...","[-0.0424, -0.0424, -0.0424, -0.0424, -0.0397, ...",0.0,0.0,False,0.0,0.0,0.0,False,0.0,120.900002,111.500000,0.86,0.8


In [4]:
# removing a row that had unusually high number of time samples
data = original_data[original_data.doorcyclecounter != 3503]

In [9]:
data = original_data[original_data['misalignment_cd_vs_ld'] != 0]
data

,doorcyclecounter,doorforce,doorspeed,doorposition,cd_counter_rollers_installation,ld_counter_rollers_installation,closing_device_failure,misalignment_cd_vs_ld,misalignment_cd_vs_ld_sill_gap,belt_tension,pulley_is_touching_belt,zero_position,cmcouplerfriction,cmdoorfriction,cmelectronicage,cmvibration
0,1166,"[-75.1, -75.1, -92.8, -73.5, -51.9, -46.3, -46...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0531, 0.078...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0451, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,154.899994,121.300003,0.86,0.9
1,1167,"[-75.1, -75.1, -85.0, -92.1, -52.8, -51.6, -51...","[0.0016, 0.0016, 0.0016, 0.0016, 0.033, 0.0822...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.048, -...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,134.399994,120.699997,0.85,0.7
2,1168,"[-75.1, -75.1, -87.9, -88.3, -61.2, -51.1, -51...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0612, 0.086...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0468, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.600006,119.800003,0.85,0.9
3,1169,"[-75.1, -75.1, -85.6, -83.4, -57.2, -51.0, -51...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0505, 0.087...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0474, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,145.800003,122.000000,0.86,0.8
4,1170,"[-75.1, -75.1, -78.5, -91.4, -59.5, -49.7, -49...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0297, 0.073...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0483, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.100006,120.400002,0.85,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,1435,"[-75.1, -75.1, -81.1, -90.6, -57.0, -48.4, -48...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0343, 0.074...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.048, -...",0.0,0.0,False,-1.0,0.0,0.0,False,0.0,148.100006,119.300003,0.86,0.8
264,1436,"[-75.1, -75.1, -88.6, -91.0, -61.8, -50.8, -50...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0492, 0.083...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0472, ...",0.0,0.0,False,-1.0,0.0,0.0,False,0.0,159.699997,120.300003,0.86,0.8
265,1437,"[-75.1, -75.1, -75.8, -93.5, -55.7, -48.6, -48...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0134, 0.068...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0489, ...",0.0,0.0,False,-1.0,0.0,0.0,False,0.0,147.699997,120.900002,0.86,0.9
266,1438,"[-75.1, -75.1, -92.3, -83.4, -58.9, -44.5, -44...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0522, 0.082...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0463, ...",0.0,0.0,False,-1.0,0.0,0.0,False,0.0,146.600006,118.699997,0.85,0.9


In [10]:
data = data.drop(['cd_counter_rollers_installation',
                                    'ld_counter_rollers_installation',
                                    'closing_device_failure',
                                    'misalignment_cd_vs_ld_sill_gap',
                                    'belt_tension',
                                    'pulley_is_touching_belt',
                                    'zero_position'], axis=1)

Defining relevant column types

In [11]:
failure_columns = [
	'cd_counter_rollers_installation',
	'ld_counter_rollers_installation',
	'closing_device_failure',
	'misalignment_cd_vs_ld',
	'misalignment_cd_vs_ld_sill_gap',
	'belt_tension',
	'pulley_is_touching_belt',
	'zero_position',
]

environment_columns = [
	'cmcouplerfriction',
	'cmdoorfriction',
	'cmelectronicage',
	'cmvibration',
]

static_state_columns = [
	'doorcyclecounter',
]

time_state_columns = [
	'doorforce',
	'doorspeed',
	'doorposition',
	# 'time',
]

In [12]:
#Data processing and analysis
cat_cols = [
	'misalignment_cd_vs_ld',
]

#make num_cols a list of all columns excluding columns that exist in cat_cols
num_cols = [col for col in data.columns if col not in cat_cols]
num_cols

['doorcyclecounter',
 'doorforce',
 'doorspeed',
 'doorposition',
 'cmcouplerfriction',
 'cmdoorfriction',
 'cmelectronicage',
 'cmvibration']

In [13]:
def sample_series(arrays, target_length):
    """
    Sample values from a series of arrays to produce arrays of equal length.
    
    Parameters:
    arrays (list of list of int/float): Series of arrays with different lengths.
    target_length (int): The length of the arrays after sampling.
    
    Returns:
    list of list of int/float: Series of arrays with the target length.
    """
    sampled_series = []

    for array in arrays:
        if len(array) == target_length:
            sampled_series.append(array)
        else:
            indices = np.linspace(0, len(array) - 1, target_length).astype(int)
            sampled_array = [array[i] for i in indices]
            sampled_series.append(sampled_array)
    
    return sampled_series

# Example usage
# arrays = [[1, 2, 3, 4, 5], [10, 20, 30], [100, 200, 300, 400, 500, 600]]
# target_length = 3
# sampled_series = sample_series(arrays, target_length)
# print(sampled_series)

In [14]:
doorforce_series = sample_series(data['doorforce'],50)
doorspeed_series = sample_series(data['doorspeed'],50)
doorposition_series = sample_series(data['doorposition'],50)

# Create new DataFrames with 'doorforce' column
doorforce_df = pd.DataFrame({'doorforce': doorforce_series})
doorspeed_df = pd.DataFrame({'doorspeed': doorspeed_series})
doorposition_df = pd.DataFrame({'doorposition': doorposition_series})

In [15]:
# Convert values into separate columns
doorforce_df = doorforce_df['doorforce'].apply(pd.Series)
doorspeed_df = doorspeed_df['doorspeed'].apply(pd.Series)
doorposition_df = doorposition_df['doorposition'].apply(pd.Series)

# Rename the columns if needed
doorforce_df.columns = [f'doorforce_{i+1}' for i in range(doorforce_df.shape[1])]
doorspeed_df.columns = [f'doorspeed_{i+1}' for i in range(doorspeed_df.shape[1])]
doorposition_df.columns = [f'doorposition_{i+1}' for i in range(doorposition_df.shape[1])]


In [16]:

truncated_data = data.copy()
truncated_data.drop(columns=['doorforce', 'doorspeed', 'doorposition'], inplace=True)
truncated_data


,doorcyclecounter,misalignment_cd_vs_ld,cmcouplerfriction,cmdoorfriction,cmelectronicage,cmvibration
0,1166,1.0,154.899994,121.300003,0.86,0.9
1,1167,1.0,134.399994,120.699997,0.85,0.7
2,1168,1.0,146.600006,119.800003,0.85,0.9
3,1169,1.0,145.800003,122.000000,0.86,0.8
4,1170,1.0,146.100006,120.400002,0.85,0.7
...,...,...,...,...,...,...
263,1435,-1.0,148.100006,119.300003,0.86,0.8
264,1436,-1.0,159.699997,120.300003,0.86,0.8
265,1437,-1.0,147.699997,120.900002,0.86,0.9
266,1438,-1.0,146.600006,118.699997,0.85,0.9


In [17]:
train_data = pd.concat([truncated_data, doorforce_df, doorspeed_df, doorposition_df], axis=1)
train_data

,doorcyclecounter,misalignment_cd_vs_ld,cmcouplerfriction,cmdoorfriction,cmelectronicage,cmvibration,doorforce_1,doorforce_2,doorforce_3,doorforce_4,...,doorposition_41,doorposition_42,doorposition_43,doorposition_44,doorposition_45,doorposition_46,doorposition_47,doorposition_48,doorposition_49,doorposition_50
0,1166,1.0,154.899994,121.300003,0.86,0.9,-75.099998,-51.900002,-48.000000,-5.5,...,0.0262,0.0095,0.0014,-0.0081,-0.0339,-0.0468,-0.0500,-0.0501,-0.0492,-0.0492
1,1167,1.0,134.399994,120.699997,0.85,0.7,-75.099998,-92.099998,-50.000000,-13.9,...,0.0311,0.0101,0.0019,-0.0068,-0.0322,-0.0483,-0.0500,-0.0501,-0.0492,-0.0492
2,1168,1.0,146.600006,119.800003,0.85,0.9,-75.099998,-88.300003,-50.200001,-12.8,...,0.0325,0.0108,0.0025,-0.0041,-0.0275,-0.0468,-0.0500,-0.0501,-0.0492,-0.0492
3,1169,1.0,145.800003,122.000000,0.86,0.8,-75.099998,-83.400002,-50.400002,-14.3,...,0.0313,0.0103,0.0019,-0.0058,-0.0295,-0.0472,-0.0501,-0.0501,-0.0495,-0.0495
4,1170,1.0,146.100006,120.400002,0.85,0.7,-75.099998,-91.400002,-49.200001,-13.0,...,0.0285,0.0106,0.0024,-0.0053,-0.0296,-0.0475,-0.0499,-0.0499,-0.0492,-0.0492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,1435,-1.0,148.100006,119.300003,0.86,0.8,-75.099998,-90.599998,-48.799999,1.3,...,0.0158,0.0057,0.0000,-0.0192,-0.0423,-0.0492,-0.0499,-0.0495,-0.0492,-0.0492
264,1436,-1.0,159.699997,120.300003,0.86,0.8,-75.099998,-91.000000,-50.400002,9.5,...,0.0220,0.0074,0.0010,-0.0063,-0.0319,-0.0474,-0.0499,-0.0499,-0.0492,-0.0492
265,1437,-1.0,147.699997,120.900002,0.86,0.9,-75.099998,-93.500000,-39.099998,-1.2,...,0.0313,0.0097,0.0021,-0.0048,-0.0296,-0.0475,-0.0499,-0.0499,-0.0492,-0.0492
266,1438,-1.0,146.600006,118.699997,0.85,0.9,-75.099998,-83.400002,-47.200001,31.5,...,0.0235,0.0102,0.0019,-0.0059,-0.0301,-0.0474,-0.0499,-0.0499,-0.0492,-0.0492


In [18]:
train_data.dropna(inplace=True, axis=0)
train_data.to_parquet("v7-truncated-50ts-f1.parquet")